# StressME for heat_evolved strain

In [1]:
import cobrame
import ecolime
import AcidifyME
import pickle
from AcidifyME import *
from AcidifyME.periplasmic_proteome import *
from AcidifyME.membrane_lipid_constraint import *
from AcidifyME.membrane_protein_activity import *
from AcidifyME.proton_influx import *
from oxidizeme.model import *

from qminospy.me1 import ME_NLP1
from cobrame.io.json import load_json_me_model


/usr/lib/python3.6/site-packages/cobra-0.5.11-py3.6-linux-x86_64.egg/cobra/solvers/__init__.py:63: UserWarning: No LP solvers found
  warn("No LP solvers found")
/usr/lib/python3.6/site-packages/cobra-0.5.11-py3.6-linux-x86_64.egg/cobra/io/sbml3.py:24: UserWarning: Install lxml for faster SBML I/O
  warn("Install lxml for faster SBML I/O")
/usr/lib/python3.6/site-packages/cobra-0.5.11-py3.6-linux-x86_64.egg/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


### To load FoldME using wild type keffs from pickle

In [2]:
with open("FoldME_Ali_keff.pickle", "rb") as f:
  me = pickle.load(f)

### To change a Keff for DXPRIi(dxr) using FoldME keff so that the strain can be heat-evolved

Supporting document:

Characterization of 1-Deoxy-d-xylulose 5-Phosphate Reductoisomerase, an Enzyme Involved in Isopentenyl Diphosphate Biosynthesis, and Identification of Its Catalytic Amino Acid Residues* (https://www.jbc.org/article/S0021-9258(19)80070-4/fulltext)

“The maximum activity was observed at 40–60 °C, and 70% of the maximum activity was obtained even at 80 °C.”
“The purified enzyme was not heat stable above 50 °C.”

In [3]:
rxn = me.reactions.get_by_id('DXPRIi_FWD_DXPREDISOM-CPLX_mod_cobalt2')
rxn.keff=88.72
rxn.update()

### To define model parameter and stress conditions

In [4]:
temperature = 37
ROS = 1.0 # fold
pH = 7.0

### To apply thermal stress

In [5]:
ecolime.chaperones.change_temperature(me, temperature)

/source/cobrame/cobrame/core/processdata.py:257 UserWarning: met (generic_tRNA_GAG_glu__L_c) does not have formula
/source/cobrame/cobrame/core/processdata.py:285 UserWarning: Negative element found in ({'H': -1}), element contributions are usually positive
/source/cobrame/cobrame/core/processdata.py:229 UserWarning: No element contribution input for subreaction (folding_KJE_1), calculating based on stoichiometry instead
/source/cobrame/cobrame/core/processdata.py:285 UserWarning: Negative element found in (defaultdict(<class 'int'>, {'H': 0, 'O': -3, 'P': -1})), element contributions are usually positive
/source/cobrame/cobrame/core/processdata.py:229 UserWarning: No element contribution input for subreaction (folding_KJE_4), calculating based on stoichiometry instead
/source/cobrame/cobrame/core/processdata.py:229 UserWarning: No element contribution input for subreaction (folding_KJE_2), calculating based on stoichiometry instead
/source/cobrame/cobrame/core/processdata.py:229 UserW

### To apply acid stress

In [6]:
fold_change=5.666666666669649*pH**4-1.396666666667337e+02*pH**3+1.286583333333885e+03*pH**2-5.254083333335287e+03*pH**1+8.037000000002481e+03
print(fold_change)

#add different constraints into the ME-model to simulate acid stress
temperatureK=273.15+temperature

add_membrane_constraint(me, 'MJR_habituated')
add_protein_stability_constraint(me, pH, temperatureK, constrain_periplasm_biomass=False)
add_proton_leak_rxn(me, pH)
modify_membrane_protein_activity(me, fold_change)

1.0


/source/cobrame/cobrame/core/reaction.py:858 UserWarning: RNA Polymerase () not found
/source/cobrame/cobrame/core/processdata.py:873 UserWarning: RNA_b3509 starts with 'AUG' which is not a start codon
/source/cobrame/cobrame/core/processdata.py:910 UserWarning: No termination enzyme for RNA_b3509


### To set up the model parameter - unmodeled_protein_fraction

The modeled_protein_fraction (90%) is close to what has been experimentally quantified for the proteome mass in E.coli (95%), and it has been increased from the previous ME models (e.g. 85% in OxidizeME) considering more proteins were added to the StressME. 

In [7]:
me.unmodeled_protein_fraction = 0.1

### To apply oxidative stress

In [8]:
solver = ME_NLP1(me)

In [9]:
stress = StressME(solver)
stress.make_stressme()

10fthf_c + 1.80324164444648e-7*mu Fmt_mono_mod_mg2_mod_k + 0.00400427350427351*mu FusA_mono + 4.27350427350428e-6*mu InfA_mono + 4.45382843794893e-6*mu InfB_mono + 4.27350427350428e-6*mu InfC_mono + 2.07386100902221e-7*mu Map_mono_mod_2:fe2 + 1.63370183870619e-7*mu PrfB_mono + 4.27350427350428e-6*mu PrfC_mono + 0.000498399634202103*mu + 0.000195123456790123 + 0.00598079561042524*(mu + 0.3915)/mu RNA_b0026 + 2.55589556001079e-8*mu + 1.00063311174422e-8 RNA_degradosome + 2.62130898398881e-7*mu Rrf_mono + 0.00400427350427351*mu Tsf_mono + 92 ala__L_c + 46 arg__L_c + 21 asn__L_c + 71 asp__L_c + 937.0 + 4.21048010973937*(mu + 0.3915)/mu atp_c + 14 cys__L_c + 0.00400427350427351*mu generic_Tuf + 48.0 generic_tRNA_AAA_lys__L_c + 18.0 generic_tRNA_AAC_asn__L_c + 13.0 generic_tRNA_AAG_lys__L_c + 3.0 generic_tRNA_AAU_asn__L_c + 2.0 generic_tRNA_ACA_thr__L_c + 32.0 generic_tRNA_ACC_thr__L_c + 6.0 generic_tRNA_ACG_thr__L_c + 7.0 generic_tRNA_ACU_thr__L_c + 9.0 generic_tRNA_AGC_ser__L_c + 4.0 gener

In [10]:
subs_dict = {
 'h2o2_c': 5e-08, 'o2s_c': 2e-10,
 #'h2o2_e': 1.5e-06, 'o2s_e': 0,
 #'pq2_c': 0.0
}

subs_dict['h2o2_c'] = 5e-08*ROS
subs_dict['o2s_c'] = 2e-10*ROS

stress.substitute_ros(solver, subs_dict)
stress.substitute_metal(solver)

In [11]:
#### Unlock ROS detox enzymes, as they might be blocked (UB=0) for some reason by default
for r in me.process_data.SPODM.parent_reactions:
    print(r.id, '\t', r.upper_bound)
    r.upper_bound = 1000
    print(r.id, '\t', r.upper_bound)
    
for r in me.process_data.SPODMpp.parent_reactions:
    print(r.id, '\t', r.upper_bound)
    r.upper_bound = 1000
    print(r.id, '\t', r.upper_bound)
    
for r in me.process_data.CAT.parent_reactions:
    print(r.id, '\t', r.upper_bound)
    r.upper_bound = 1000
    print(r.id, '\t', r.upper_bound)
    
me.reactions.MOX_REV_CPLX_dummy.upper_bound = 0

SPODM_FWD_SUPEROX-DISMUTMN-CPLX_mod_mn2 	 0
SPODM_FWD_SUPEROX-DISMUTMN-CPLX_mod_mn2 	 1000
SPODM_FWD_SUPEROX-DISMUTFE-CPLX_mod_fe3 	 0
SPODM_FWD_SUPEROX-DISMUTFE-CPLX_mod_fe3 	 1000
SPODMpp_FWD_G6886-MONOMER_mod_cu2 	 0
SPODMpp_FWD_G6886-MONOMER_mod_cu2 	 1000
CAT_FWD_HYDROPEROXIDI-CPLX_mod_pheme 	 0
CAT_FWD_HYDROPEROXIDI-CPLX_mod_pheme 	 1000
CAT_FWD_HYDROPEROXIDII-CPLX_mod_6:hemed 	 0
CAT_FWD_HYDROPEROXIDII-CPLX_mod_6:hemed 	 1000


In [12]:
stress.force_o2s_from_pq()

for r in me.reactions.query('PQ2RED'):
    print(r.id, '\t', r.lower_bound)

PQ2RED_FWD_FLAVONADPREDUCT-MONOMER_mod_fad 	 0.26606684


### To run simualtions from Warm Start

Warm start file is basis.npy, which was saved from previous successful simulation. The warm start can signifiantly reduce the simulation time

In [13]:
%%time
saved_basis = np.load('basis.npy')
MU_PREC = 1e-04
sol, hs, xopt, cache  = solver.bisectmu(precision=MU_PREC, basis = saved_basis)    

iter	muopt    	a     	b     	mu1       	stat1
Finished compiling expressions in 244.258295 seconds
Finished substituting S,lb,ub in 17.103582 seconds
Finished makeME_LP in 2.772035 seconds
Getting MINOS parameters from ME_NLP...
1 1.0 1.0 2.0 1.0 optimal
Finished substituting S,lb,ub in 16.731938 seconds
Finished makeME_LP in 2.776474 seconds
Getting MINOS parameters from ME_NLP...
2 1.0 1.0 1.5 1.5 1
Finished substituting S,lb,ub in 16.777529 seconds
Finished makeME_LP in 2.777131 seconds
Getting MINOS parameters from ME_NLP...
3 1.0 1.0 1.25 1.25 1
Finished substituting S,lb,ub in 16.800883 seconds
Finished makeME_LP in 2.772342 seconds
Getting MINOS parameters from ME_NLP...
4 1.0 1.0 1.125 1.125 1
Finished substituting S,lb,ub in 16.759459 seconds
Finished makeME_LP in 2.766798 seconds
Getting MINOS parameters from ME_NLP...
5 1.0 1.0 1.0625 1.0625 1
Finished substituting S,lb,ub in 16.876287 seconds
Finished makeME_LP in 2.783848 seconds
Getting MINOS parameters from ME_NLP...
6 1

### Results and save as sol.pickle

In [14]:
# solution status
print(me.solution.status)
# growth rate
print(me.reactions.biomass_dilution.x)
# to save solution to pickle
# write solution file 
filename = 'StressME_heatevolved'+'_T_'+str(temperature)+'_pH_'+str(pH)+'_ROS_'+str(ROS)+'X'
with open(filename+'_sol.pickle','wb') as f:
    pickle.dump(me.solution,f)

optimal
1.04156494140625
